# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [9]:
# imports
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [6]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [10]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

openai = OpenAI()

API key found and looks good so far!


In [11]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
messages=[
        {"role": "system", "content": "You are a helpful programming assistant."},
        {"role": "user", "content": question}
    ]

In [12]:
# Get gpt-4o-mini to answer, with streaming
stream = openai.chat.completions.create(model=MODEL_GPT,messages=messages,stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)
    

The line of code you provided is using Python functionality to generate a sequence of values in a specific way. Let's break it down:

1. **Set Comprehension**: The part inside the curly braces `{...}` is a set comprehension. This creates a set of unique authors from a collection of books. The expression looks like this:
   
   python
   {book.get("author") for book in books if book.get("author")}
   

   - `books` is presumably an iterable (like a list or another collection) containing book dictionaries.
   - `book.get("author")` retrieves the value associated with the key `"author"` in each book dictionary.
   - The `if book.get("author")` condition filters out any books that do not have an author (i.e., where the author key might not exist or its value is `None`, `''`, etc.).

   As a result, this set comprehension will produce a set of unique author names for all books that do have an author.

2. **`yield from` Statement**: The `yield from` expression is used to yield all values from an iterable. In this case, it yields each author from the created set.

   python
   yield from { ... }
   

   This suggests that the code is part of a generator function. When this generator function is called, it will yield each element from the set of authors one by one.

### Summary:
In short, this line of code collects unique authors from a list of book dictionaries and yields them one by one. The use of `yield from` allows for lazy iteration over the unique authors without having to store them in a list explicitly.

### Example:
Suppose you have the following `books` list:

python
books = [
    {'title': 'Book One', 'author': 'Author A'},
    {'title': 'Book Two', 'author': 'Author B'},
    {'title': 'Book Three', 'author': 'Author A'},
    {'title': 'Book Four'},  # No author
    {'title': 'Book Five', 'author': None},  # No author
]


The set comprehension would produce `{'Author A', 'Author B'}`. The `yield from` would let the generator function yield 'Author A' and then 'Author B' separately when iterated.

In [15]:
# Get Llama 3.2 to answer
import ollama
OLLAMA_BASE_URL = "http://localhost:11434/v1"

response = ollama.chat(model=MODEL_LLAMA, messages=messages)

reply = response['message']['content']
display(Markdown(reply))

**Code Explanation**

The given code uses Python's `yield from` syntax to generate an iterator that yields values from a generator expression.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's a breakdown of what the code does:

1. `{...}` is a dictionary comprehension, which is equivalent to a list comprehension but returns a dictionary instead.
2. `for book in books` iterates over each item (`book`) in the `books` collection ( likely a list or other iterable).
3. `if book.get("author")` filters out any items that don't have an "author" key present in the dictionary.
4. Inside the comprehension, `.get("author")` retrieves the value associated with the "author" key for each remaining item (`book`). This will return `None` if no such key is found.

**Why it's used**

The purpose of this code is to efficiently generate an iterator that yields author names from a collection of books. By using a generator expression, you can avoid loading all author names into memory at once, which can be beneficial when working with large datasets.

Here's how it works:

* When the `yield from` statement is executed, the dictionary comprehension is evaluated first.
* For each item in the `books` collection, the corresponding author name is retrieved and added to the dictionary (which is essentially a list).
* The iterator yields one value at a time, which are the author names.

**Example Use Case**

Suppose you have a list of book dictionaries and you want to extract their authors:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"}
]

for author in yield from {book.get("author") for book in books if book.get("author")}>
    print(author)
```

This code will output:

```
Author A
Author B
```